# Cake Eating 2, Numerical Approach

That last one was great, but for any model that's not CRRACE (Constant Relative Risk Aversion Cake Eating), there won't be analytical solutions. As such, we should use our one analytical example to compare our numerical alternatives (which we could then apply to other problems).

Let's started by installing a necessary package:

In [1]:
#!pip install interpolation

Then let's pull in the package libraries we'll need:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#from interpolation import interp
from scipy.optimize import minimize_scalar, bisect
from scipy.interpolate import LinearNDInterpolator
from scipy import stats
from scipy.optimize import minimize


### Reviewing Findings
The value function (for any cake eating problem) is given by:
$$
v(x)=\max_{\{0\leq c\leq x_t\}}u(c)+\beta v(x-c) \quad \forall \ x\geq0
$$

We found the analytical solutions to be given by:

In Computing the Bellman Operator, we need to use a maximization algorithm, so let's build a quick one in Python:

In [3]:
def maximize(g, args):

    objective = lambda x: -g(x, *args) #define obj function
    #minimize the negative
    result = minimize_scalar(objective)
    #define output
    maximizer, maximum = result.x, -result.fun


    #return
    return maximizer, maximum

#Example of maximize in action
#Note that you *do not* specify the first argument of the function 
  #(because that's what maximize() will check over)
def f(x,a,c):
  return a*(x-c)**2

#print(f(0,-1,0))
        #f, a, b,  a , c
#maximize(f,-10,10,([-1,0],[0,1]))

Let's build a class, called `CakeEating` to store $\beta$ and $\gamma$ as well as a `state_action_value` method which computes the utility for a certain consumption choice and guessed value function.

But first let's take a detour to discuss classes in Python.



In [4]:

class YouTube:

    def __init__(self,
                 disc=0.96,       # discount factor
                 α=.1,             # alg importance of previous streams
                 β=.1,             # alg importance of quantity
                 γ=.1,             # alg importance of shock
                 δ=.7,            # audience depreciation per period
                 κ=20,             # Talent
                 q=1,            # Cost of quantity over quality
                 r=1,             # royalty per stream
                 n_t_1_range=[1e-4,5],  # exclude zero for numerical stability
                 A_t_1_range=[1e-4,1000],   # maximum number of pieces in a period
                 e_dist=stats.norm(), #distribution of epsilon
                 m_range=[1e-3,10],
                 Niters=12, # number of sample points for each state variable
                 ):

        #NGL, I have no idea what this actually does
        self.disc, self.α, self.β, self.γ, self.δ, self.κ, self.q, self.r = disc, α, β, γ, δ, κ, q, r

        # Set up grid (of input values)
        self.Niters= Niters
        self.e_dist = e_dist
        self.n_t_1_range, self.A_t_1_range = n_t_1_range, A_t_1_range
        self.n_t_1_grid = np.linspace(n_t_1_range[0], n_t_1_range[1], Niters)
        self.A_t_1_grid = np.linspace(A_t_1_range[0], A_t_1_range[1], Niters)
        self.e_quantiles = np.linspace(1/(Niters+1),Niters/(Niters+1),Niters)
        #self.e_quantiles = np.linspace(1/(100+1),100/(100+1),100)
        self.e_grid = e_dist.ppf(self.e_quantiles)
        self.m_grid = np.linspace(m_range[0], m_range[1], Niters)

        self.mm, self.nn, self.AA, self.ee = np.meshgrid(self.m_grid,self.n_t_1_grid,self.A_t_1_grid,self.e_grid)
      
        #create a grid of where v could actually be eval-ed at
        self.n_t_possibilities=self.n(self.z(self.mm))
        self.A_t_possibilities=self.A(self.mm,self.nn,self.AA,self.ee)
        self.u_possibilities=self.u(self.mm,self.nn,self.AA,self.ee)


    def z(self,m): #relate z and m through f^-1
      q = self.q
      out = self.κ*m**(-q)
      return out

    def n(self,z): # Streams per quality once exposed
      n_max=self.n_t_1_range[1]
      n = (1-np.exp(-z*1))*n_max
      return np.clip(n,self.n_t_1_range[0],self.n_t_1_range[1])

    def I(self,m,n_t_1,e):
      return -(n_t_1*self.α)-(m*self.β)-(e*self.γ)

    def A(self,m,n_t_1,A_t_1,e):
      c=(self.A_t_1_range[1]-(1-self.δ)*A_t_1)/((1-self.δ)*A_t_1)
      A= self.A_t_1_range[1]/(1+c*np.exp(self.I(m,n_t_1,e)))
      return np.clip(A,self.A_t_1_range[0],self.A_t_1_range[1])
  


    # Expected per Period
    # Utility function
    def u(self, m,n_t_1,A_t_1,e):
      disc, α, β, γ, δ, κ, q, r = self.disc, self.α, self.β, self.γ, self.δ, self.κ, self.q, self.r
      A = self.A
      n = self.n
      z = self.z
      return r*A(m,n_t_1,A_t_1,e)*n(z(m))

    # UNUSED RIGHT NOW-----
    # first derivative of utility function. (done algebraically)
    #def u_prime(self, c):

        #return c ** (-self.γ)
 
    # END UNUSED -----


    def v_func(self, v_array):
      nn, AA = np.meshgrid(self.n_t_1_grid,self.A_t_1_grid)


      # coorce into vector of R^2 coordinates
      eval_coords= np.array((nn,AA)).T.reshape(-1,2)

      # Create an interpolant
      v = LinearNDInterpolator(eval_coords, v_array.reshape(-1))

      return v


    def E_state_action_value_fast(self, m,n_t_1,A_t_1,v_func):
      """
      Right hand side of the Bellman equation given x and c.
      """

      #call the utility function and beta from the object
      u, disc = self.u, self.disc
      A = self.A

    # Define the v function which uses linear interpolation
        # recall we ran: "from interpolation import interp"
        # lambda allows us to define a function on the spot
          # since it's "lambda": x, x is the input 
          #(because v_arry and x_grid are passed from before)

        # 2 2d matrices of where to evaluate the state variables at
      
        # v looks for 3 inputs so v(n,A,e) gives guess of value fn

      v=v_func


      e_quantiles = self.e_quantiles
        #so v(x) returns the interpolated value of x_grid,v_array at x
        #return the utility you get now u(c), plus the 
        #guess at optimal utility play given x-c consumption 
        #This is the RHS of Bellman equation


        # how much density lives between eval points
      weight_vec = e_quantiles-np.concatenate(([0],e_quantiles[:-1]))

        # where the eval points of epsilon are
      e_eval_points = self.e_grid

      mm, nn, AA, ee = np.meshgrid(m,n_t_1,A_t_1,e_eval_points)
        # evaluate at State Action Val each eval point
        #SAV=self.state_action_value(m,n_t_1,A_t_1,e_eval_points,v_array)

      n_t=self.n(self.z(mm))
      A_t=A(mm,nn,AA,ee)
      U=u(mm,nn,AA,ee) + disc * v(n_t,A_t)
      E_SAV_out=np.tensordot(weight_vec,U,axes =((0),(3)))

      return E_SAV_out.flatten()

In [5]:
def T(v, yt):
    """
    The Bellman operator.  Updates the guess of the value function (array).

    * ce is an instance of CakeEating
    * v is an array representing a guess of the value function

    """
    # make an empty vector with the same shape as v
    v_new = np.empty_like(v)
    m_new = np.empty_like(v)

    #enumerate make i the counter and allows x to be the value
    #making lists of categoricals nice to loop over

                          #note that we pull the x.grid from the object 'ce'
                          #of class CakeEating that is passed to T(v,ce)
                          #We do not use CakeEating().x_grid


    # Here iv is a *vector* of indexes that allows us to loop over 
    # the multiple dimensional array of state variables
    # element is the actual vector of parameters themselves (not indexes)

    nn, AA = np.meshgrid(yt.n_t_1_grid,yt.A_t_1_grid)
    v_current = yt.v_func(v)

    for iv, element in np.ndenumerate(nn):
        #print(iv)
        # Maximize RHS of Bellman equation at state x
            #maximize ce's state action value function
        
        #Note that an x in ce.x_grid corresponds to the size of the pie at
          # present, so the max pie we could consume now x 
            #(first x in maximize)
          #And the size of present pie determines v(x-c),
            #(second x in maximize)

        # We defined maximize as follows: def maximize(g, a, b, args):
        # where g is the objective function
        # a is the start, b is the end points,
        # and args are the arguments for g, the obj function 
                            #note you do not specify consumption
                            #bc maximize is using that argument to max over
                            #g                      #a    #b  #args

        
        m_new[iv], v_new[iv]  = maximize(yt.E_state_action_value_fast, (nn[iv], AA[iv], v_current))
        #maximize returns "maximizer, maximum", 
          #so we select the *second* item, maximum

    return v_new, m_new

In [6]:
def compute_value_function(ce, # an object of class CakeEating
                           tol=1e-4, # the stopping tolerance
                           max_iter=1000, # the bailout number of iters
                           verbose=True, # the thing to run with/w/o messages
                           print_skip=25, # frequency of printout
                           v0=None): 
    # Set up loop
    if v0 is None:
        v = np.ones((len(ce.n_t_1_grid),len(ce.A_t_1_grid)))
    else:
        v=v0
     # Initial guess, just the zero function
    i = 0 #starting iterator
    error = tol + 1 # starting value of tolerance

    #run loop as long as error is too big and haven't gone too many times
    while i < max_iter and error > tol:
        v_new, m_update = T(v, ce) #bellman operate

        #this is a bad way of computing error (should probably be RMSE)
        error = np.max(np.abs(v - v_new))
        
        #error = np.sqrt(np.mean((v-v_new)**2))
        i += 1 #note this way of incrementing

        # If at a print step, print current error
        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")
        # update v to new guess
        v = v_new
    # if we reach bailout, bailout
    if i == max_iter:
        print("Failed to converge!")
    # if converged and verbose, say so
    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    # return convergent value function (or the last one at bailout)
    return v_new, m_update

## Model Implementation

In [7]:
# yt=YouTube(α=.1,
#            β=.1,
#            γ=.1,
#            δ=.70,
#            κ=20,
#            Niters=20,
#            e_dist=stats.norm(scale=1))

# #yt.A_grid=np.array([1e-4,])
# #v_array,m_array=compute_value_function(yt)
# nn, AA = np.meshgrid(yt.n_t_1_grid,yt.A_t_1_grid)

In [8]:
# from google.colab import files
# np.savetxt('v_jump.csv', v_array, delimiter=',') 
# files.download('v_jump.csv')
# np.savetxt('m_jump.csv', m_array, delimiter=',') 
# files.download('m_jump.csv')

In [9]:
# from google.colab import files
# np.savetxt('nn_20.csv', nn, delimiter=',') 
# files.download('nn_20.csv')
# np.savetxt('AA_20.csv', AA, delimiter=',') 
# files.download('AA_20.csv')

In [10]:
def simulate(yt,v_array,m_array,T_steps,N_agents,n0=None,A0=None):

  m_func=yt.v_func(m_array)
  v_func=yt.v_func(v_array)

  if n0 is None:
    n0 = yt.n_t_1_range[0]
  if A0 is None:
    A0 = yt.A_t_1_range[0]


  # Create n matrix and initialize first row
  n = np.empty((T_steps,N_agents))
  n[0,:] = n0

  # Same thing for A
  A = np.zeros_like(n)
  A[0,:] = A0

  # Matrix of per period revenues:
  u = np.empty_like(n)

  # Matrix of expected lifetime revenue at start of any period:
  v = np.empty_like(n)

  # Matrix of strategies used each peirod:
  m = np.empty_like(n)

  # Pregenerate shocks
  e = yt.e_dist.rvs((T_steps,N_agents))

  # Generate T matrix
  T=np.repeat(np.arange(0,T_steps).reshape((-1,1)),N_agents,axis=1)
  # Generate id matrix
  ids=np.repeat(np.arange(0,N_agents).reshape((1,-1)),T_steps,axis=0)

  for i in range(n.shape[0]-1): # each row is a time step
    v[i,:] = v_func(n[i,:],A[i,:])
    m[i,:] = m_func(n[i,:],A[i,:])
    A[i+1,:]=yt.A(m[i,:],n[i,:],A[i,:],e[i,:])
    n[i+1,:]=yt.n(yt.z(m[i,:]))
    u[i,:] = yt.u(m[i,:],n[i,:],A[i,:],e[i,:])
  
  return m , v , A , n , u, T, ids

In [11]:
yt_k20=YouTube(κ=20,Niters=24)
yt_k10=YouTube(κ=10,Niters=24)
yt_d30=YouTube(δ=.3,Niters=24)
yt_sd5=YouTube(e_dist=stats.norm(scale=5),Niters=24)
v_array_k20,m_array_k20=compute_value_function(yt_k20)
v_array_k10,m_array_k10=compute_value_function(yt_k10)
v_array_d30,m_array_d30=compute_value_function(yt_d30)
v_array_sd5,m_array_sd5=compute_value_function(yt_sd5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in reciprocal


Error at iteration 25 is 324.90563932159785.
Error at iteration 50 is 42.20062679072362.
Error at iteration 75 is 5.481261897504737.
Error at iteration 100 is 0.7119380513831857.
Error at iteration 125 is 0.09247063930160948.
Error at iteration 150 is 0.012010623435344314.
Error at iteration 175 is 0.001560009433887899.
Error at iteration 200 is 0.0002026306901825592.

Converged in 209 iterations.
Error at iteration 25 is 262.4030630711968.
Error at iteration 50 is 34.08243006962584.
Error at iteration 75 is 4.426823474761477.
Error at iteration 100 is 0.5749814798109583.
Error at iteration 125 is 0.07468192577653099.
Error at iteration 150 is 0.009700121197965927.
Error at iteration 175 is 0.0012599079491337761.
Error at iteration 200 is 0.00016364425391657278.

Converged in 207 iterations.
Error at iteration 25 is 563.190266508449.
Error at iteration 50 is 73.15045128786733.
Error at iteration 75 is 9.501241030571691.
Error at iteration 100 is 1.2340707681069034.
Error at iteration 1

In [17]:
np.random.seed(314159)
out_k20 = simulate(yt_k20,v_array_k20,m_array_k20,100,1000,n0=1e-4)
out_k10 = simulate(yt_k10,v_array_k10,m_array_k10,100,1000,n0=1e-4)
out_d30 = simulate(yt_d30,v_array_d30,m_array_d30,100,1000,n0=1e-4)
out_sd5 = simulate(yt_sd5,v_array_sd5,m_array_sd5,100,1000,n0=1e-4)

In [18]:
from google.colab import files
np.savetxt('sim_k20.csv', np.array(out_k20).reshape((7,-1)).T, delimiter=',') 
files.download('sim_k20.csv')
np.savetxt('sim_k10.csv', np.array(out_k10).reshape((7,-1)).T, delimiter=',') 
files.download('sim_k10.csv')
np.savetxt('sim_d30.csv', np.array(out_d30).reshape((7,-1)).T, delimiter=',') 
files.download('sim_d30.csv')
np.savetxt('sim_sd5.csv', np.array(out_sd5).reshape((7,-1)).T, delimiter=',') 
files.download('sim_sd5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
np.random.seed(314159*2)
out_k20 = simulate(yt_k20,v_array_k20,m_array_k20,100,1000,n0=5)
out_k10 = simulate(yt_k10,v_array_k10,m_array_k10,100,1000,n0=5)
out_d30 = simulate(yt_d30,v_array_d30,m_array_d30,100,1000,n0=5)
out_sd5 = simulate(yt_sd5,v_array_sd5,m_array_sd5,100,1000,n0=5)
np.max(out_k20[2][1:,:])

472.6397480135764

In [20]:
from google.colab import files
np.savetxt('sim_k20_n5.csv', np.array(out_k20).reshape((7,-1)).T, delimiter=',') 
files.download('sim_k20_n5.csv')
np.savetxt('sim_k10_n5.csv', np.array(out_k10).reshape((7,-1)).T, delimiter=',') 
files.download('sim_k10_n5.csv')
np.savetxt('sim_d30_n5.csv', np.array(out_d30).reshape((7,-1)).T, delimiter=',') 
files.download('sim_d30_n5.csv')
np.savetxt('sim_sd5_n5.csv', np.array(out_sd5).reshape((7,-1)).T, delimiter=',') 
files.download('sim_sd5_n5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>